# Ukrainian Stories For Kids Generation Based On Multilingual Bert

The goal of this final project was to train multilingual Bert from Google on Ukrainian corpus to compare original model with trained version on Masked Language Model and Next Sentence Prediction combined to see how good the original model was and if some improvement could have been made.

One of the biggest challenges that was faced in this project was to find a suitable dataset. Since Ukrainian corpuses are not widespread it was necessary to create one. The initial guess was that although BERT is claiming to be multilingual, it was not performing well on low-resource languages like Ukrainian. The assumption proved itself to be true as you will be able to see later. Short stories for kids and fairytales are a good candidates for training corpus since they are comprised of not so big of a voabulary and generally have similar narration structure. It's important to mention that vocabulary of a child is not as developed as that of an adult, so the model might do a much better job training on it.

In [17]:
!pip install pytorch-nlp
!pip install tokenize_uk
!pip install pytorch_transformers

In [0]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from pytorch_transformers import *
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import torch
import tokenize_uk
import os
% matplotlib inline

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
print("Found GPU at: {}".format(device_name))

Found GPU at: /device:GPU:0


In [21]:
torch.cuda.is_available()

True

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
device

device(type='cuda')

# Part 1: Alteration based on Trained Masked Language Model

The first part of the project focuses on altering stories based on Masked Language Model. The input for this part is comprised of three stories for kids. Each story is broken down into sentences and in each sentence one word is picked by random and is masked by [MASK] token. It's importnat to note that in reality we might have a compound word that, if masked, would be masked by multiple [MASK] tokens, but for the sake of the project each word, regardless of the size, was masked by one [MASK] token. The idea is to see what the original BERT might plaace in place of the masked token and compare the results to the trained version to see which one understands Ukrainian language better.

In [0]:
import torch
from pytorch_transformers import *
from pytorch_transformers.tokenization_bert import BertTokenizer
from random import random, randrange, randint, shuffle, choice

In [0]:
# Masks one word at random by a single [MASK] token in a given sentence
def mask_token_in_sentence(sentence):
  tokenized_sentence = tokenize_uk.tokenize_words(sentence)
  number_of_tokens = len(tokenized_sentence)
  # print(tokenized_sentence, number_of_tokens)
  masked_token = False
  while (not masked_token):
    # print(number_of_tokens)
    index = randrange(number_of_tokens)
    # print(index)
    # To eliminate tokenization of punctioation like .,;:
    delims = {", ", ".", "!", ":", "?", "'", ";", ''}
    if (not tokenized_sentence[index] in delims):
      tokenized_sentence[index] = "[MASK]"
      masked_token = True
  # print(tokenized_sentence)
  reconstructed_sentence = " ".join(tokenized_sentence)
  return reconstructed_sentence

In [0]:
# Returns a set of unique punctuation marks in a given texts
def get_unique_delimiters(texts):
  delimiters = set()
  for text in texts:
    for word in tokenize_uk.tokenize_words(text):
      if (len(word) == 1 and not word in delimiters and not word.isalpha() and not word.isdigit()):
        delimiters.add(word)
  return delimiters

Please Note: you need to have a corresponding folder with fairytales on your google drive to load the data. Refer to the GitHub repositrory to download it.

In [0]:
first_text = ""
with open("/content/drive/My Drive/Fairytales/test/Тхір і лис.txt", 'r') as f:
  first_text = f.read()

second_text = ""
with open ("/content/drive/My Drive/Fairytales/test/Чудесні бички.txt", 'r') as f:
  second_text = f.read()
  
third_text = ""
with open ("/content/drive/My Drive/Fairytales/test/Їжачок та дівчинка.txt", "r") as f:
  third_text = f.read()

In [0]:
# texts = [first_text, second_text, third_text]

In [0]:
# unique_delimiters = get_unique_delimiters(texts)
# print(unique_delimiters)
# print(len(unique_delimiters))

In [0]:
# Cleans texts by substituting similar punctuation marks with a single standard
def clean_texts(texts):

  dashes = {'–', '—', '―', '~'} # replace with -
  special_symbols = {'№', '_', '<', '>', '|', ']', '*', '[', '^', '&'} # replace with ""
  apostrophes = {'’', '‘'} # replace with '
  direct_speech = {'“', '»', '«'} # replace with '"'
  three_dots = {'…'} # replace with '.

  counter = 0
  for i in range(len(texts)):
    print("Processing text: ", i)
    text = texts[i]
    words = []
    tokenized_words = tokenize_uk.tokenize_words(text)
    for word in tokenized_words:
      added = False

      for dash in dashes:
        if (dash in word):
          new_word = word.replace(dash, "-")
          words.append(new_word)
          added = True
          continue

      for special_symbol in special_symbols:
        if(special_symbol in word):
          new_word = word.replace(special_symbol, "")
          words.append(new_word)
          added = True
          continue
      
      for apostrophe in apostrophes:
        if (apostrophe in word):
          new_word = word.replace(apostrophe, "'")
          words.append(new_word)
          added = True
          continue

      for direct in direct_speech:
        if (direct in word):
          new_word = word.replace(direct, '"')
          words.append(new_word)
          added = True
          continue
      
      for dots in three_dots:
        if (dots in word):
          counter += 1
          new_word = word.replace(dots, '.')
          words.append(new_word)
          added = True
          continue
      if (not added):
        words.append(word)
    reconstructed_text = " ".join(words)
    texts[i] = reconstructed_text 
  return texts

In [0]:
# texts = clean_texts(texts)

In [0]:
# Preprares text for sentence tokenization by tokenize_uk
def prepare_for_tokenization(texts):
  new_texts = []
  for i in range(len(texts)):
    text = texts[i]
    text = text.replace("?", "?.")
    text = text.replace("!", "!.")
    text = text.replace(":", ":.")
    text = text.replace(". -", ". ")
    new_texts.append(text)
  return new_texts

In [0]:
# new_texts = prepare_for_tokenization(texts)
# print("Number of texts: ", len(new_texts))

In [0]:
# Tokenizes texts to sentences
def tokenize_texts_to_sentences(texts):
  text_sentences = []
  for text in texts:
    text_to_add = []
    text_to_add += tokenize_uk.tokenize_sents(text)
    text_sentences.append(text_to_add)
  return text_sentences

In [0]:
# text_sentences = tokenize_texts_to_sentences(new_texts)
# print("Number of tokenized texts: ", len(text_sentences))

In [0]:
# for i in range(len(text_sentences)):
#   print(i, " contains ", len(text_sentences[i]), " sentences")

In [0]:
# print(new_texts[0])

In [0]:
# Cleans sentences after tokenization
def clean_after_tokenization(text_sentences):
  cleaned = []

  for j in range(len(text_sentences)):
    text_to_work = text_sentences[j]
    for i in range(len(text_to_work)):
      sentence = text_to_work[i]
      sentence = sentence.replace("?.", "?")
      sentence = sentence.replace("!.", "!")
      sentence = sentence.replace(":.", ":")
      text_to_work[i] = sentence
    cleaned.append(text_to_work)
  return cleaned

In [0]:
# text_sentences = clean_after_tokenization(text_sentences)

In [0]:
# for i in range(len(text_sentences)):
#   print(i, " contains ", len(text_sentences[i]), " sentences")

In [0]:
# Masks words in sentences
def mask_tokens(text_sentences):
  masked = []
  for j in range(len(text_sentences)):
    text_to_work = text_sentences[j]
    for i in range(len(text_to_work)):
      sentence = text_to_work[i]
      # print(sentence)
      sentence = mask_token_in_sentence(sentence)
      text_to_work[i] = sentence
    masked.append(text_to_work)
  return masked

In [0]:
# text_sentences = mask_tokens(text_sentences)

In [0]:
# for i in range(len(text_sentences)):
#   print(i, " contains ", len(text_sentences[i]), " sentences")

In [0]:
# print("Number of texts: ", len(text_sentences))

In [0]:
# for i in range(len(text_sentences[0])):
#   sentence = text_sentences[0][i]
#   sentence = sentence.replace("[CLS] ", "")
#   sentence = sentence.replace("[SEP] ", "")
#   text_sentences[0][i] = sentence

In [0]:
# for sent in text_sentences[0]:
#   print(sent)

In [0]:
# Loading tokenizer from traineed BERT
tokenizer_path = "/content/drive/My Drive/BERT/finetuned_for_inference/vocab.txt"
trained_tokenizer = BertTokenizer(tokenizer_path, do_lower_case=False)

In [47]:
# Loading tokenizer from original BERT
basic_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)

100%|██████████| 995526/995526 [00:00<00:00, 1993057.27B/s]


In [0]:
# text_sentences[0]

In [49]:
# Loads trained BertFroPreTraining model
model_state_dict_path = "/content/drive/My Drive/BERT/finetuned_for_inference/pytorch_model.bin"
model_folder_path = "/content/drive/My Drive/BERT/finetuned_for_inference"

model_state_dict = torch.load(model_state_dict_path,  map_location='cpu')
trained_model = BertForPreTraining.from_pretrained(model_folder_path, state_dict=model_state_dict)
trained_model.eval()
trained_model.to(device)

BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIn

In [50]:
# Loads basic BertForPreTraining model
basic_model = BertForPreTraining.from_pretrained('bert-base-multilingual-cased')
basic_model.eval()
basic_model.to(device)

100%|██████████| 714314041/714314041 [00:25<00:00, 28199757.21B/s]


BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): BertIn

In [0]:
# trained_model.to(device)

In [0]:
# Returns segment ids and indexed tokens for a given text
def get_segments_and_tokens(text_input, given_tokenizer):
  text = text_input
  tokenized_text = []
  indexed_tokens = []
  segment_ids = []

  print("Number of sentences in text: ", len(text))
  longest_sentence = 0
  for sent in text:
    if (len(sent) > longest_sentence):
      longest_sentence = len(sent)
  print("Longest sentence: ", longest_sentence)

  for i in range(len(text)):
    text[i] = "[CLS] " + text[i] + " [SEP]" 

  for i in range(len(text)):
    tokenized_text.append(given_tokenizer.tokenize(text[i]))
    # print(tokenized_text[i])
    indexed_tokens.append(given_tokenizer.convert_tokens_to_ids(tokenized_text[i]))

  indexed_tokens = pad_sequences(indexed_tokens, maxlen=longest_sentence, dtype="long", truncating="post", padding="post")
  print("First indexed tokens length: ", len(indexed_tokens[0]))

  for i in range(len(text)):
    segment = []
    num_tokens = len(indexed_tokens[i])
    segment_ids.append([0] * num_tokens)

  print("Segment ids length: ", len(segment_ids))
  print("First segment ids length: ", len(segment_ids[0]))

  return segment_ids, indexed_tokens

  # segments_tensor = (torch.tensor(segment_ids))
  # tokens_tensor = torch.tensor(indexed_tokens)

  # return segments_tensor, tokens_tensor

In [0]:
# Changes [MASK] tokens from predictions by choosing word with highest probability
def make_inference(predictions, inference):
  # predictions = masked_lm_logits_scores

  # inference = indexed_tokens
  new_inference = []
  for i in range(len(inference)):
    # print(i)
    tokens = inference[i]
    # print(tokens)
    for j in range(len(tokens)):
      if (tokens[j] == 103):
        # print(103)
        token_to_insert = torch.argmax(predictions[i][j]).item()
        tokens[j] = token_to_insert
    # inference[i] = tokens
    new_inference.append(tokens)
  return new_inference

In [0]:
# Convertes inferred tokens to text and cleans it
def convert_inference_to_sentences(inference, given_tokenizer):
  predicted_sentences = []
  for x in range(len(inference)):
    first_sentence = given_tokenizer.convert_ids_to_tokens(inference[x])
    first_sentence = given_tokenizer.convert_tokens_to_string(first_sentence)
    first_sequence = first_sentence.replace("[PAD]", "")
    first_sequence = first_sequence.replace("[CLS] ", "")
    first_sequence = first_sequence.replace(" [SEP]", "")
    predicted_sentences.append(first_sequence)
  return predicted_sentences

In [0]:
# Generates Masked Language Model Result for a given text with given tokenizer and given model
def generate_results(text_sentences, given_model, given_tokenizer):
    predicted_texts = []

    for i in range(len(text_sentences)):
        print("Processing text: ", i)
        text = text_sentences[i]
        segment_ids, indexed_tokens = get_segments_and_tokens(text, given_tokenizer)
        print("Recieved segments and indexed_tokens")

        segments_tensor = (torch.tensor(segment_ids))
        tokens_tensor = torch.tensor(indexed_tokens)

        print("Model started")

        input_length = len(segments_tensor)
        print(input_length)

        logits = []
        text = []
        for j in range(0, input_length, 10):

            left = j
            right = j + 10
            if (right > input_length):
                right = input_length

            if (left == input_length - 1):
                right = input_length

            segments_portion = segment_ids[left:right]
            tokens_portion = indexed_tokens[left:right]
            segments_tensor = (torch.tensor(segments_portion))
            tokens_tensor = torch.tensor(tokens_portion)

            tokens_tensor = tokens_tensor.to(device)
            segments_tensor = segments_tensor.to(device)

            print("Processing from {} to {}".format(left, right))
            with torch.no_grad():
                masked_lm_logits_scores, _ = given_model(tokens_tensor, segments_tensor)
                # logits.append(masked_lm_logits_scores)

            # logits = np.stack(logits, axis = 0)
            print("Model finished")

            predictions = masked_lm_logits_scores
            inference = indexed_tokens[left:right]

            new_inference = make_inference(predictions, inference)
            print("Recieved new inference")

            predicted_sentences = convert_inference_to_sentences(new_inference, given_tokenizer)
            print("Recieved predicted_sentences")

            text += predicted_sentences

        predicted_texts.append(text)
        print("Appended new sentences for: ", i)
        print("\n")

    saved_predicted = predicted_texts

    cleaned = []
    for j in range(len(saved_predicted)):
        text_to_work = saved_predicted[j]
        for i in range(len(text_to_work)):
            sentence = text_to_work[i]
            sentence = sentence.replace(" ?", "?")
            sentence = sentence.replace(" !", "!")
            sentence = sentence.replace(" :", ":")
            sentence = sentence.replace("  ", "")
            sentence = sentence.replace(" ,", ",")
            sentence = sentence.replace(" .", ".")
            text_to_work[i] = sentence
        cleaned.append(text_to_work)

    cleaned_texts = cleaned
    # cleaned_texts = []
    # for clean in cleaned:
    #     cleaned_texts.append(" ".join(clean))

    return cleaned_texts

In [0]:
    texts = [first_text, second_text, third_text]

    unique_delimiters = get_unique_delimiters(texts)
    print(unique_delimiters)
    print(len(unique_delimiters))

    texts = clean_texts(texts)

    new_texts = prepare_for_tokenization(texts)
    print("Number of texts: ", len(new_texts))

    text_sentences = tokenize_texts_to_sentences(new_texts)
    print("Number of tokenized texts: ", len(text_sentences))

    for i in range(len(text_sentences)):
        print(i, " contains ", len(text_sentences[i]), " sentences")

    print(new_texts[0])

    text_sentences = clean_after_tokenization(text_sentences)

    for i in range(len(text_sentences)):
        print(i, " contains ", len(text_sentences[i]), " sentences")

    text_sentences = mask_tokens(text_sentences)

    for i in range(len(text_sentences)):
        print(i, " contains ", len(text_sentences[i]), " sentences")

    print("Number of texts: ", len(text_sentences))

    for sent in text_sentences[0]:
        print(sent)

    cleaned_trained = generate_results(text_sentences, trained_model, trained_tokenizer)

    for j in range(len(text_sentences)):
        current = text_sentences[j]
        for i in range(len(current)):
          sentence = current[i]
          sentence = sentence.replace("[CLS] ", "")
          sentence = sentence.replace(" [SEP]", "")
          current[i] = sentence
        text_sentences[j] = current

    cleaned_basic = generate_results(text_sentences, basic_model, basic_tokenizer)

# Part 1: Results

Below are the sentences with masked words and predictions based on original and trained models. Again, it might be the case that the word would be masked by multiple tokens as in the original implementation, but for the sake of this project any word, regardless of size, was masked by exactly one [MASK] token to see how two models would perform.

From the inference we can see some interesting results that need to be discussed. First of all, unfortunately, original BERT does a poor job on masked words. For example, in sentence 

*'[CLS] Раз прийшов лис до [MASK] в гості , та й тхір його гарно погостив . [SEP]'* 

original BERT outputs 

*'Раз прийшов лис доu в гості, та й тхір його гарно погостив.'* 
and in 

*'[CLS] Той чоловік пригонить бички додому та й [MASK] : [SEP]'* 
the output was 

*'Той чоловік пригонить бички додому та йо:'*.

As we can see, original BERT not only fails to predict correct part of speech, but even fails to make predictions in ukrainian as we can see from the first sentence. This is a clear proof of the fact that the corpus that was used to train Multilingual Bert for Ukrainian didn't generalize language well enough. This can be attributed to the fact that Ukrainian is a low-resource language and it's hard to create a good corpus. It's also possible that due to the fact that multilingual BERT was trained on numerous languages it would take a lot of time to gather a good corpus for each language. This gives an opportunity for researchers from different countries to impove BERT as it's easier for them to gather corpus specifc to their region.

On the contrary, trained BERT gives promising results. In the same sentences:

'[CLS] Раз прийшов лис до [MASK] в гості , та й тхір його гарно погостив . [SEP]' and '[CLS] Той чоловік пригонить бички додому та й [MASK] : [SEP]' trained model outputs:

'Раз прийшов лис до нього в гості, та й тхір його гарно погостив.' and  'Той чоловік пригонить бички додому та й каже:'

These are pretty good results considering the fact that the original word was masked with only one MASK token. **As we can see, the model not only identifes the correct part of speech, but also outputs pronoun in the right gender.** This is due to the fact that BERT usses attention mechanis to infer context from surrounding words and thus, as a result, understands the context better and can make pretty good predictions.

However, even trained BERT sometimes fails to understand the context. It might be noticed that the model might place punctuation marks like ",", or "." and "-" in place of the actual word. This result is not surprising since the corpus that the model was trained on consisted of many dialogues with those particular punctuation marks.

<u> As a result, we can see that even when the model is trained on a small corpus the model outputs much better reults comparing to that of the original model. </u>

In [73]:
text_sentences[0]

['[CLS] Раз прийшов лис до [MASK] в гості , та й тхір його гарно погостив . [SEP]',
 '[CLS] Але приходить [MASK] тхір до лиса , а лис каже : [SEP]',
 '[CLS] Не маю я , куме [MASK] чим вас погостити , хіба би пішли зі мною тут до одного ґазди , у нього є цілий курник . [SEP]',
 '[CLS] [MASK] вони обоє і пішли шукати курей . [SEP]',
 '[CLS] Прийшли вони до того [MASK] . [SEP]',
 '[CLS] Каже [MASK] до тхора : [SEP]',
 '[CLS] Тут [MASK] курник . [SEP]',
 '[CLS] Ану , лізьте цією діркою досередини , а я буду на варті ; лиш собі , - каже , - не жалійте [MASK] бо господаря не шкода , у нього є досталь ! [SEP]',
 '[CLS] Тхір поліз досередини , лис заклав [MASK] дошкою , а сам підбіг під вікно та крикнув : [SEP]',
 '[CLS] [MASK] ґаздо ! [SEP]',
 '[CLS] [MASK] ! [SEP]',
 '[CLS] А ідіть - но до [MASK] , маєте зло\xadдія . [SEP]',
 '[CLS] Та й [MASK] . [SEP]',
 '[CLS] Ґазда [MASK] , збудив синів та й повибігали надвір . [SEP]',
 '[CLS] [MASK] крик ! [SEP]',
 "[CLS] А той тхір уже собі гарно під'їв

In [68]:
cleaned_trained[0]

['Раз прийшов лис до нього в гості, та й тхір його гарно погостив.',
 'Але приходить до тхір до лиса, а лис каже: ',
 'Не маю я, куме, чим вас погостити, хіба би пішли зі мною тут до одного ґазди, у нього є цілий курник.',
 'Всі вони обоє і пішли шукати курей. ',
 'Прийшли вони до того міста.',
 'Каже - до тхора:',
 'Тут - курник. ',
 'Ану, лізьте цією діркою досередини, а я буду на варті ; лиш собі, - каже, - не жалійте, бо господаря не шкода, у нього є досталь! ',
 'Тхір поліз досередини, лис заклав під дошкою, а сам підбіг під вікно та крикнув:',
 '- ґаздо!',
 '-! ',
 'А ідіть - но до того, маєте злодія.',
 'Та й.. ',
 'Ґаздачі, збудив синів та й повибігали надвір. ',
 '- крик! ',
 "А той тхір уже собі гарно під ' їв та й уже учув, що біда, а він далі.",
 'Штовхнув дошку та й береться лізти, а то не получається, бо став дуже товстий.',
 'А ті як взяли бука, як стануть бити, то тхір ледве, через дірку й утік ледве живий. ',
 'Від того часу тхір з лисом у великій ненависті.']

In [74]:
cleaned_basic[0]

['Раз прийшов лис доu в гості, та й тхір його гарно погостив.',
 'Але приходить до тхір до лиса, а лис каже: ',
 'Не маю я, кумев чим вас погостити, хіба би пішли зі мною тут до одного ґазди, у нього є цілий курник.',
 '- вони обоє і пішли шукати курей. ',
 'Прийшли вони до того..',
 'Каже - до тхора:',
 'Тут " курник. ',
 'Ану, лізьте цією діркою досередини, а я буду на варті ; лиш собі, - каже, - не жалійте, бо господаря не шкода, у нього є досталь! ',
 'Тхір поліз досередини, лис заклав під дошкою, а сам підбіг під вікно та крикнув:',
 'on ґаздо!',
 'on! ',
 'А ідіть - но до., маєте злодія.',
 'Та йself. ',
 'Ґазда., збудив синів та й повибігали надвір. ',
 'on крик! ',
 "А той тхір уже собі гарно під ' їв та й само учув, що біда, а він далі.",
 'Штовхнув дошку та й береться лізти, а то не получається, бо став дуже товстий.',
 'А ті як взяли бука, як стануть бити, то тхір ледве, через дірку й утік ледве живий. ',
 'Від того часу тхір з лисом. великій ненависті.']

In [75]:
text_sentences[1]

['[CLS] Був один чоловік бідний і любив [MASK] колись трош\xadки напиватися . [SEP]',
 '[CLS] А до того бив дуже свою [MASK] . [SEP]',
 '[CLS] Але вона йому [MASK] не казала , лиш каже : [SEP]',
 '[CLS] Хай тобі пан Бог заплатить за мене , - а [MASK] нічого . [SEP]',
 '[CLS] Аж [MASK] разу каже той чоловік : [SEP]',
 '[CLS] Ану , я [MASK] до Бога за зарплатою . [SEP]',
 '[CLS] І [MASK] у ліс . [SEP]',
 '[CLS] Іде лісом , іде , дивиться : ангел летить з неба та [MASK] каже : [SEP]',
 '[CLS] А куди ти , чоловіче [MASK] ідеш ? [SEP]',
 '[CLS] [MASK] до Бога за зарплатою . [SEP]',
 '[CLS] [MASK] якою зарплатою ? питається ангел . [SEP]',
 '[CLS] А що ж , - каже той [MASK] , - я бив свою жінку , а вона все казала : " Хай тобі пан Бог заплатить ! " А я тепер іду до Бога , аби мені заплатив . [SEP]',
 '[CLS] Ангел [MASK] : [SEP]',
 '[CLS] Ходи сюди , [MASK] . [SEP]',
 '[CLS] І повів його за корч , а там пара бичків пасеться [MASK] і каже ангел : [SEP]',
 '[CLS] [MASK] тобі цю пару бичків та й

In [76]:
cleaned_trained[1]

['Був один чоловік бідний і любивший колись трошки напиватися. ',
 'А до того бив дуже свою..',
 'Але вона йому, не казала, лиш каже: ',
 'Хай тобі пан Бог заплатить за мене, - а. нічого.',
 'Аж, разу каже той чоловік: ',
 'Ану, я, до Бога за зарплатою. ',
 'І - у ліс. ',
 'Іде лісом, іде, дивиться: ангел летить з неба та й каже:',
 'А куди ти, чоловіче, ідеш? ',
 '- до Бога за зарплатою. ',
 '- якою зарплатою? питається ангел. ',
 'А що ж, - каже той чоловік, - я бив свою жінку, а вона все казала: " Хай тобі пан Бог заплатить! " А я тепер іду до Бога, аби мені заплатив.',
 'Ангел -: ',
 'Ходи сюди, а. ',
 'І повів його за корч, а там пара бичків пасеться, і каже ангел:',
 '- тобі цю пару бичків та й іди додому. ',
 '- сам пішов до неба. ',
 'Той чоловік пригонить бички додому та й каже:',
 'Дав мені - Бог пару бичків. ',
 'А в того чоловіка мав брат такий сильний багач, що пари йому не було на ціле село. ',
 'Той багач був дуже лукавий і не любив свого брата, та й пішов до міста.',
 '

In [77]:
cleaned_basic[1]

['Був один чоловік бідний і любивy колись трошки напиватися. ',
 'А до того бив дуже свою..',
 'Але вона йому mai не казала, лиш каже: ',
 'Хай тобі пан Бог заплатить за мене, - а не нічого.',
 'Аж tale разу каже той чоловік: ',
 'Ану, я, до Бога за зарплатою. ',
 'І. у ліс. ',
 'Іде лісом, іде, дивиться: ангел летить з неба та, каже:',
 'А куди ти, чоловіче - ідеш? ',
 '" до Бога за зарплатою. ',
 ', якою зарплатою? питається ангел. ',
 'А що ж, - каже той же, - я бив свою жінку, а вона все казала: " Хай тобі пан Бог заплатить! " А я тепер іду до Бога, аби мені заплатив.',
 'Ангел ": ',
 'Ходи сюди, m. ',
 'І повів його за корч, а там пара бичків пасеться, і каже ангел:',
 ', тобі цю пару бичків та й іди додому. ',
 'a сам пішов до неба. ',
 'Той чоловік пригонить бички додому та йо:',
 'Дав мені " Бог пару бичків. ',
 'А в того чоловіка був брат такий сильний багач, що пари йому не було на ціле село. ',
 'Той багач був дуже лукавий і не любив свого брата, та й пішов до..',
 'А тоді т

In [78]:
text_sentences[2]

['[CLS] Жив [MASK] у лісі маленький їжачок . [SEP]',
 '[CLS] Була у нього своя [MASK] . [SEP]',
 '[CLS] Збирав [MASK] яблучка , грибочки , ягоди . [SEP]',
 '[CLS] Тільки було йому дуже [MASK] . [SEP]',
 '[CLS] От одного разу вирішив він знайти собі [MASK] . [SEP]',
 '[CLS] Вийшов він із хатинки [MASK] й пішов до лісу . [SEP]',
 '[CLS] Блукав їжачок дуже довго , але тільки ніхто не захотів [MASK] ним товаришувати . [SEP]',
 '[CLS] Лисичка сказала , що він [MASK] колючий . [SEP]',
 '[CLS] Зайчик сказав [MASK] що їжачок швидко не бігає і з ним йому буде не цікаво , а ведмідь зовсім не захотів з ним розмовляти . [SEP]',
 '[CLS] Розплакався [MASK] та й пішов собі стежкою . [SEP]',
 '[CLS] Ішов [MASK] , ішов та й дійшов до людського саду . [SEP]',
 '[CLS] Бачить , [MASK] там яблучок на землю багато - багато нападало . [SEP]',
 '[CLS] От їжачок і подумав : " Назбираю я [MASK] собі додому " . [SEP]',
 '[CLS] Тільки почав [MASK] , аж раптом почув тріск сухої гілки за спиною . [SEP]',
 '[CLS] Ко

In [79]:
cleaned_trained[2]

['Жив, у лісі маленький їжачок.',
 'Була у нього своя..',
 'Збирав, яблучка, грибочки, ягоди. ',
 'Тільки було йому дуже.. ',
 'От одного разу вирішив він знайти собі.. ',
 'Вийшов він із хатинки та й пішов до лісу.',
 'Блукав їжачок дуже довго, але тільки ніхто не захотів з ним товаришувати.',
 'Лисичка сказала, що він був колючий.',
 'Зайчик сказав, що їжачок швидко не бігає і з ним йому буде не цікаво, а ведмідь зовсім не захотів з ним розмовляти.',
 'Розплакався, та й пішов собі стежкою. ',
 'Ішов та, ішов та й дійшов до людського саду. ',
 'Бачить, що там яблучок на землю багато - багато нападало.',
 'От їжачок і подумав: " Назбираю я - собі додому ".',
 'Тільки почав час, аж раптом почув тріск сухої гілки за спиною.',
 'Коли їжачок обернувся, побачив маленьку дівчинку і дуже великі.',
 'Але дівчинка й не збиралася на його.',
 'Вона могла назбирати яблучок їжачкові на голочки. ',
 'Він дуже було і запропонував дівчинці погратися. ',
 'Довго вони гралися, доки дівчинку не було мати

In [80]:
cleaned_basic[2]

['Жив. у лісі маленький їжачок.',
 'Була у нього своя ".',
 'Збирав. яблучка, грибочки, ягоди. ',
 'Тільки було йому дуже ". ',
 'От одного разу вирішив він знайти собі.. ',
 'Вийшов він із хатинки. й пішов до лісу.',
 'Блукав їжачок дуже довго, але тільки ніхто не захотів с ним товаришувати.',
 'Лисичка сказала, що він був колючий.',
 'Зайчик сказав, що їжачок швидко не бігає і з ним йому буде не цікаво, а ведмідь зовсім не захотів з ним розмовляти.',
 'Розплакався. та й пішов собі стежкою. ',
 'Ішов та, ішов та й дійшов до людського саду. ',
 'Бачить, " там яблучок на землю багато - багато нападало.',
 'От їжачок і подумав: " Назбираю я, собі додому ".',
 'Тільки почавt, аж раптом почув тріск сухої гілки за спиною.',
 'Коли їжачок обернувся, побачив маленьку дівчинку і дуже..',
 'Але дівчинка й не збиралася o його.',
 'Вона може назбирати яблучок їжачкові на голочки. ',
 'Він дуже lent і запропонував дівчинці погратися. ',
 'Довго вони гралися, доки дівчинку не - мати.',
 'Але перед 

In [0]:
# predicted_texts = []

# for i in range(len(text_sentences)):
#   print("Processing text: ", i)
#   text = text_sentences[i]
#   segment_ids, indexed_tokens = get_segments_and_tokens(text)
#   print("Recieved segments and indexed_tokens")

#   segments_tensor = (torch.tensor(segment_ids))
#   tokens_tensor = torch.tensor(indexed_tokens)

#   print("Model started")

#   input_length = len(segments_tensor)
#   print(input_length)

#   logits = []
#   text = []
#   for j in range(0, input_length, 10):

#     left = j
#     right = j+10
#     if (right > input_length):
#       right = input_length

#     if (left == input_length - 1):
#       right = input_length

#     segments_portion = segment_ids[left:right]
#     tokens_portion = indexed_tokens[left:right]
#     segments_tensor = (torch.tensor(segments_portion))
#     tokens_tensor = torch.tensor(tokens_portion)

#     tokens_tensor = tokens_tensor.to(device)
#     segments_tensor = segments_tensor.to(device)

#     print("Processing from {} to {}".format(left, right))
#     with torch.no_grad():
#         masked_lm_logits_scores, _ = maskedLM_model(tokens_tensor, segments_tensor)
#         # logits.append(masked_lm_logits_scores)
    
#   # logits = np.stack(logits, axis = 0)
#     print("Model finished")

#     predictions = masked_lm_logits_scores
#     inference = indexed_tokens[left:right]

#     new_inference = make_inference(predictions, inference)
#     print("Recieved new inference")

#     predicted_sentences = convert_inference_to_sentences(new_inference)
#     print("Recieved predicted_sentences")

#     text += predicted_sentences


#   predicted_texts.append(text)
#   print("Appended new sentences for: ", i)
#   print("\n")

In [0]:
# saved_predicted = predicted_texts

In [0]:
# cleaned = []
# for j in range(len(saved_predicted)):
#     text_to_work = saved_predicted[j]
#     for i in range(len(text_to_work)):
#       sentence = text_to_work[i]
#       sentence = sentence.replace(" ?", "?")
#       sentence = sentence.replace(" !", "!")
#       sentence = sentence.replace(" :", ":")
#       sentence = sentence.replace("  ", "")
#       sentence = sentence.replace(" ,", ",")
#       sentence = sentence.replace(" .", ".")
#       text_to_work[i] = sentence
#     cleaned.append(text_to_work)

In [0]:
# cleaned_texts = []
# for clean in cleaned:
#   cleaned_texts.append(" ".join(clean))

In [0]:
# cleaned_texts[0]

In [0]:
# text_sentences[0]

In [0]:
# cleaned_texts[1]

In [0]:
# text_sentences[1]

In [0]:
# cleaned_texts[2]

In [0]:
# text_sentences[2]

# Part 2: Story Generation Based On Next Sentence Prediction

The second part of the project was primiraly focused on how good of a story can BERT generated given outputs from the previous part. This is definitely a hard task for the model since its inputs are outputs of the trained model from the previous step. This means that the context of the stories from the previous step might be somewhat obscured and thus lead to not so surprising results.

As mentioned, the inputs for this part are the outputs of the trained model from the previous step. Here we again break each story into sentences and choose first sentence of the first story as our foundation. After that we cansider each pair of sentences consisting of our first sentene and all other sentences. After this we choose a pair with the highest probability of the second sentence being a continuation of the first one which gives us a second sentence. The process now repeats with the second sentence to generate the third. The length of the story is: MIN(length of the 1st story, length of the 2nd + 3rd stories). This is a computation heavy task, so GPU might be a good choice for this. 

In [0]:
# from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification, BertForNextSentencePrediction, BertForMaskedLM, BertForPreTraining
from pytorch_transformers.tokenization_bert import BertTokenizer
from random import random, randrange, randint, shuffle, choice

In [0]:
prepared_second = []
for clean in cleaned_trained:
  prepared_second.append(" ".join(clean))

In [0]:
first_text = prepared_second[0]
second_text = prepared_second[1]
third_text = prepared_second[2]

In [0]:
texts = [first_text, second_text, third_text]

In [0]:
# Cleans texts by substituting similar punctuation marks with a single standard
def clean_texts(texts):

  dashes = {'–', '—', '―', '~'} # replace with -
  special_symbols = {'№', '_', '<', '>', '|', ']', '*', '[', '^', '&'} # replace with ""
  apostrophes = {'’', '‘'} # replace with '
  direct_speech = {'“', '»', '«'} # replace with '"'
  three_dots = {'…'} # replace with '.

  counter = 0
  for i in range(len(texts)):
    print("Processing text: ", i)
    text = texts[i]
    words = []
    tokenized_words = tokenize_uk.tokenize_words(text)
    for word in tokenized_words:
      added = False

      for dash in dashes:
        if (dash in word):
          new_word = word.replace(dash, "-")
          words.append(new_word)
          added = True
          continue

      for special_symbol in special_symbols:
        if(special_symbol in word):
          new_word = word.replace(special_symbol, "")
          words.append(new_word)
          added = True
          continue
      
      for apostrophe in apostrophes:
        if (apostrophe in word):
          new_word = word.replace(apostrophe, "'")
          words.append(new_word)
          added = True
          continue

      for direct in direct_speech:
        if (direct in word):
          new_word = word.replace(direct, '"')
          words.append(new_word)
          added = True
          continue
      
      for dots in three_dots:
        if (dots in word):
          counter += 1
          new_word = word.replace(dots, '.')
          words.append(new_word)
          added = True
          continue
      if (not added):
        words.append(word)
    reconstructed_text = " ".join(words)
    texts[i] = reconstructed_text 
  return texts

In [90]:
texts = clean_texts(texts)

Processing text:  0
Processing text:  1
Processing text:  2


In [0]:
# Prepares sentences for tokenization
def prepare_for_tokenization(texts):
  new_texts = []
  for i in range(len(texts)):
    text = texts[i]
    text = text.replace("?", "?.")
    text = text.replace("!", "!.")
    text = text.replace(":", ":.")
    text = text.replace(". -", ". ")
    new_texts.append(text)
  return new_texts


In [92]:
new_texts = prepare_for_tokenization(texts)
print("Number of texts: ", len(new_texts))

Number of texts:  3


In [0]:
# Tokenizes texts into sentences 
def tokenize_texts_to_sentences(texts):
  text_sentences = []
  for text in texts:
    text_to_add = []
    text_to_add += tokenize_uk.tokenize_sents(text)
    text_sentences.append(text_to_add)
  return text_sentences

In [94]:
text_sentences = tokenize_texts_to_sentences(new_texts)
print("Number of tokenized texts: ", len(text_sentences))

Number of tokenized texts:  3


In [95]:
for i in range(len(text_sentences)):
  print(i, " contains ", len(text_sentences[i]), " sentences")

0  contains  16  sentences
1  contains  45  sentences
2  contains  25  sentences


In [0]:
# print(new_texts[0])

In [0]:
# Cleans texts after splitting them to sentences
def clean_after_tokenization(text_sentences):
  cleaned = []

  for j in range(len(text_sentences)):
    text_to_work = text_sentences[j]
    for i in range(len(text_to_work)):
      sentence = text_to_work[i]
      sentence = sentence.replace("?.", "?")
      sentence = sentence.replace("!.", "!")
      sentence = sentence.replace(":.", ":")
      text_to_work[i] = sentence
    cleaned.append(text_to_work)
  return cleaned

In [0]:
text_sentences = clean_after_tokenization(text_sentences)

In [99]:
for i in range(len(text_sentences)):
  print(i, " contains ", len(text_sentences[i]), " sentences")

0  contains  16  sentences
1  contains  45  sentences
2  contains  25  sentences


In [0]:
# Computes generated story length
def get_story_length(text_sentences):
  story_length = len(text_sentences[0])
  length_of_stories = len(text_sentences[1]) + len(text_sentences[2])
  if (story_length > length_of_stories):
    story_length = length_of_stories
  return story_length

In [0]:
# Creates list of sentences for consideration
def get_sentences_for_consideration(text_sentences):
  sentences_for_considerartion = text_sentences[0][1:]
  sentences_for_considerartion += text_sentences[1]
  sentences_for_considerartion += text_sentences[2]

  return sentences_for_considerartion

In [0]:
# Returns segment ids and preprared sentence pairs
def get_segment_ids_and_prepared_text(sentences_for_considerartion):

  prepared_text = []
  segment_ids = []
  for j in range(len(sentences_for_considerartion)):
    first_sentence = "[CLS] " + story[current_sentence_index] + " [SEP] " 
    second_sentence = sentences_for_considerartion[j] + " [SEP] " 
    segment_ids.append([0]* len(first_sentence) + [1]*len(second_sentence))
    prepared_text.append(first_sentence + second_sentence)
  
  return segment_ids, prepared_text

In [0]:
# Returns length of longest sentence pair
def get_longest_sequence(prepared_text):
  longest_sentence = 0
  for sent in prepared_text:
      if (len(sent) > longest_sentence):
        longest_sentence = len(sent)
  return longest_sentence

In [0]:
# Returns indexed tokens and segment ids
def get_indexed_tokens_and_segment_ids(prepared_text, segment_ids):
  indexed_tokens = []
  segment_ids = segment_ids

  for j in range(len(prepared_text)):
    tokenized_text.append(trained_tokenizer.tokenize(prepared_text[j]))
    indexed_tokens.append(trained_tokenizer.convert_tokens_to_ids(tokenized_text[j]))
  
  indexed_tokens = pad_sequences(indexed_tokens, maxlen=longest_sentence, dtype="long", truncating="post", padding="post")
  segment_ids = pad_sequences(segment_ids, maxlen=longest_sentence, dtype="long", truncating="post", padding="post")

  return indexed_tokens, segment_ids

In [0]:
# Returns index for sentence from sentences_for_consideraton to add to story
def get_index_for_adding_sentence(logits):
  real_logits = []
  for logit in logits:
    real_logits.append([logit[0], logit[1]])

  real_logits = np.asarray(real_logits)

  indices = []
  logits_list = []
  for i in range(real_logits.shape[0]):
    if (np.argmax(real_logits[i]) == 0):
      indices.append(i)
      logits_list.append(real_logits[i][0]) # first trial showed that it's better of without abs function!
  
  if (len(logits_list) == 0):
    index = np.argmax(logits, axis = 0)[0]
  else:
    index = indices[np.argmax(logits_list)]
  return index

In [109]:
story = [text_sentences[0][0]]
current_sentence_index = 0

story_length = get_story_length(text_sentences)
print("Story length: ", story_length)
sentences_for_considerartion = get_sentences_for_consideration(text_sentences)
print("Number of sentences for consideration: ", len(sentences_for_considerartion))

Story length:  16
Number of sentences for consideration:  85


In [110]:
for i in range(current_sentence_index, story_length-1):
  
  tokenized_text = []
  indexed_tokens = []
  prepared_text = []
  segment_ids = []

  print("Current sentence index: ", current_sentence_index)
  segment_ids, prepared_text = get_segment_ids_and_prepared_text(sentences_for_considerartion)

  longest_sentence = get_longest_sequence(prepared_text)
  print("Longest sentence is: ", longest_sentence)
  
  indexed_tokens, segment_ids = get_indexed_tokens_and_segment_ids(prepared_text, segment_ids)

  input_length = len(segment_ids)
  logits = []
  step = 10

  for j in range(0, input_length, step):

    left = j
    right = j+step

    if (right > input_length):
      right = input_length

    if (left == input_length - 1):
      right = input_length

    segments_portion = segment_ids[left:right]
    tokens_portion = indexed_tokens[left:right]
    segments_tensor = (torch.tensor(segments_portion))
    tokens_tensor = torch.tensor(tokens_portion)

    segments_tensor = segments_tensor.to(device)
    tokens_tensor = tokens_tensor.to(device)

    print("Processing from {} to {}".format(left, right))
    with torch.no_grad():
        _, seq_relationship_logits = trained_model(tokens_tensor, segments_tensor)
        logits += list(seq_relationship_logits.cpu().numpy())
  
  index = get_index_for_adding_sentence(logits)
  print(index)
  print("Adding sentence: ", sentences_for_considerartion[index])
  story.append(sentences_for_considerartion[index])

  del sentences_for_considerartion[index]

  print("Number of sentences for consideration: ", len(sentences_for_considerartion))

  current_sentence_index += 1

Current sentence index:  0
Longest sentence is:  322
Processing from 0 to 10
Processing from 10 to 20
Processing from 20 to 30
Processing from 30 to 40
Processing from 40 to 50
Processing from 50 to 60
Processing from 60 to 70
Processing from 70 to 80
Processing from 80 to 85
5
Adding sentence:  Тут - курник .
Number of sentences for consideration:  84
Current sentence index:  1
Longest sentence is:  270
Processing from 0 to 10
Processing from 10 to 20
Processing from 20 to 30
Processing from 30 to 40
Processing from 40 to 50
Processing from 50 to 60
Processing from 60 to 70
Processing from 70 to 80
Processing from 80 to 84
45
Adding sentence:  Так йому Бог допоміг , що він одної ночі все зробив .
Number of sentences for consideration:  83
Current sentence index:  2
Longest sentence is:  309
Processing from 0 to 10
Processing from 10 to 20
Processing from 20 to 30
Processing from 30 to 40
Processing from 40 to 50
Processing from 50 to 60
Processing from 60 to 70
Processing from 70 to 8

In [114]:
story

['Раз прийшов лис до нього в гості , та й тхір його гарно погостив .',
 'Тут - курник .',
 'Так йому Бог допоміг , що він одної ночі все зробив .',
 'Та й ..',
 'Ангел - :',
 'Їжачок дуже зрадів і погодився .',
 'За годину заволочив і ..',
 'Ходи сюди , а .',
 'Дав мені - Бог пару бичків .',
 'І - у ліс .',
 'А та гора надi водою .',
 'Всі вони обоє і пішли шукати курей .',
 'Аж , разу каже той чоловік :',
 'Але приходить до тхір до лиса , а лис каже :',
 'Каже - до тхора :',
 'Тільки було йому дуже ..']

In [0]:
cleaned = []
for j in range(len(story)):
      sentence = story[j]
      sentence = sentence.replace(" ?", "?")
      sentence = sentence.replace(" !", "!")
      sentence = sentence.replace(" :", ":")
      sentence = sentence.replace("  ", "")
      sentence = sentence.replace(" ,", ",")
      sentence = sentence.replace(" .", ".")
      cleaned.append(sentence)
    # cleaned.append(text_to_work)
# story = " ".join(cleaned)

In [116]:
story

['Раз прийшов лис до нього в гості , та й тхір його гарно погостив .',
 'Тут - курник .',
 'Так йому Бог допоміг , що він одної ночі все зробив .',
 'Та й ..',
 'Ангел - :',
 'Їжачок дуже зрадів і погодився .',
 'За годину заволочив і ..',
 'Ходи сюди , а .',
 'Дав мені - Бог пару бичків .',
 'І - у ліс .',
 'А та гора надi водою .',
 'Всі вони обоє і пішли шукати курей .',
 'Аж , разу каже той чоловік :',
 'Але приходить до тхір до лиса , а лис каже :',
 'Каже - до тхора :',
 'Тільки було йому дуже ..']

# Part 2: Results

While this was an interesting experiment, unfortunately, the trained model did a poor of comprising a good story. One of the reasons might be that the input for this step was the output of the trained model from the last step, but some masks were just substituted by punctuation marks and so it was rather hard to infer a context from it. Another reason for this performance might be that the task of new story generation is wrong in its assumption. The assumption was that the stories for kids and fairytales are similar in context and so something new and interesting might be generated from it. As we can see, the model didn't give any meaningful results, so it's possible that when stories are picked in a way that their characters and plots are similar, the model might do a much better job of comprising a new story based on Next Sentence Prediction.

# Conclusions

Ukrainian Stories For Kids Generation Based On Multilingual Bert was definitely an interesting and educating project. Throughout the development the concepts of Transformers and Attention were learned. This blog was particularly useful for visualization and understanding of basic concepts:
http://jalammar.github.io.

One of the biggest challenges was to formulate the objective and to choose the right model. Ukrainian language was chosen for a reason since a lot less Data Scientists contribute to it and it imposes more obstacles than English or Russian. 

The dataset of fairytales and children stories was hand-picked, downloaded and saved in order to train the model. This particualr dataset was used because one of the main assumptions was that stories for children have much smaller vocabulary and simpler language constructions, and thus the model after training might yield better results.

As we can see from the Part 1, trained model does a better job identifiying masked tokens than the original multilingual BERT. It's been pointed out that original multilingual BERT sometimes outputs words or letters from incorrect language, which proves the relevance of this final project. Part 2 results, however, are not so shiny since the model had a tough job predicting the next sentence. This might be improved by gathering a bigger dataset and creating a bigger corpus for the model to learn from.


All in all, this project included a lot of interesting ideas, details, steps and techniques, some of which are:


*   Gathering and using completely new dataset and not relying on an existing one
*   Using state-of-the art BERT model with transformer and attention mechanisms
*   Formulating non-trivial task in terms of NLP
*   Choosing Ukrainian as it's harder and imposes more obstacles than English


This project required not only knowledge from DataRoot University, but also understanding of frontier ML concepts and thus led to very interesting outcomes.



